# LLM Response Analysis Explorer

This notebook is part pf the `repetiprompter` project.  It provides an interactive exploration of LLM response quality and convergence patterns. It's designed for researchers, engineers, and data scientists who want to understand how language models behave across multiple iterations of recursive improvement. What began as a curiosity led-exploration into recursive prompting has widened into a deeper analysis.

## What We'll Analyze Here

1. **Structural Stats**: Response length, vocabulary diversity, repetition patterns
2. **Quality Signals**: Coherence, information density, specificity, alignment, and hedging
3. **Convergence Analysis**: How responses drift or stabilize over time
4. **Interactive Exploration**: Filter and drill down into interesting patterns

Each metric helps identify strengths, weaknesses, or patterns in the generated text.

## Getting Started

First, let's ensure we have all necessary dependencies installed. (the project itself uses UV for this)

In [ ]:
# Install required packages if needed
import sys
import subprocess

def install_if_missing(package, import_name=None):
    if import_name is None:
        import_name = package
    try:
        __import__(import_name)
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Core dependencies
install_if_missing("pandas")
install_if_missing("matplotlib")
install_if_missing("seaborn")
install_if_missing("numpy")
install_if_missing("ipywidgets")

# Optional dependencies for enhanced analysis
try:
    import spacy
    HAS_SPACY = True
except ImportError:
    HAS_SPACY = False
    print("\nspaCy not found. For enhanced entity recognition, install with:")
    print("pip install spacy && python -m spacy download en_core_web_sm")

try:
    import sentence_transformers
    HAS_EMBEDDINGS = True
except ImportError:
    HAS_EMBEDDINGS = False
    print("\nSentence transformers not found. For semantic analysis, install with:")
    print("pip install sentence-transformers")

print("\nDependencies checked!")

## Setup & Data Loading

Let's import our analysis modules and load a sample run.

In [ ]:
# Path / system setup
from pathlib import Path
import sys
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Add the repo root to Python path for imports
repo_root = Path.cwd().parent
sys.path.insert(0, str(repo_root))

# Analysis modules
from json_analyzers import convergence_tracker as ct
from json_analyzers import quality_signals as qs
from json_analyzers import structural_stats as ss

# Interactive widgets
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

# Set up plotting style
plt.style.use('default')
sns.set_palette("viridis")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("Modules imported successfully!")

In [ ]:
# Auto-detect and load the most recent JSONL file
runs_dir = repo_root / "runs"

jsonl_file = None
df_nodes = None

if runs_dir.exists():
    jsonl_files = list(runs_dir.glob("*.jsonl"))
    if jsonl_files:
        # Sort by modification time, get the most recent
        jsonl_file = max(jsonl_files, key=lambda f: f.stat().st_mtime)
        print(f"Loading: {jsonl_file.name}")
        
        # Load nodes into a DataFrame for inspection
        with open(jsonl_file, 'r') as f:
            nodes = [json.loads(line) for line in f if line.strip()]
        
        df_nodes = pd.DataFrame(nodes)
        print(f"\nLoaded {len(df_nodes)} responses")
        print(f"Run ID: {df_nodes['run_id'].iloc[0]}")
        print(f"Model: {df_nodes['model_name'].iloc[0]}")
        print(f"Temperature: {df_nodes['temperature'].iloc[0]}")
        
        # Show first few rows
        df_nodes.head()
    else:
        print("No JSONL files found in runs/ directory")
else:
    print("Runs directory not found. Please ensure you're in the correct location.")

# Guard clause - stop if no data loaded
if jsonl_file is None:
    raise ImportError("\n\n⚠️ NO DATA LOADED: Please ensure you have JSONL files in the runs/ directory before proceeding.")

## 1. Structural Statistics

Structural stats give us the basic shape of our responses - how long they are, how diverse the vocabulary is, and whether they're getting repetitive.

In [ ]:
# Run structural stats analysis
if jsonl_file is None:
    print("⚠️ Cannot run analysis - no data loaded")
else:
    struct_results = ss.analyze_structural_stats(jsonl_file)
    df_struct = pd.DataFrame(struct_results)

# Display summary with highlighting
if 'df_struct' not in locals():
    print("⚠️ No structural data available - please run the previous cell first")
else:
    def highlight_extremes(df):
        styled = df.copy()
        # Highlight max values in red, min in blue
        for col in ['response_words', 'vocab_diversity', 'repetition_score']:
            if col in df.columns:
                styled[col] = df[col].apply(
                    lambda x: f'**{x:.3f}**' if x == df[col].max() else 
                    f'*{x:.3f}*' if x == df[col].min() else f'{x:.3f}'
                )
        return styled

    display(Markdown("### Structural Statistics Summary"))
    display(df_struct.style.format({
        'response_words': '{:.0f}',
        'vocab_diversity': '{:.3f}',
        'repetition_score': '{:.3f}',
        'avg_sentence_length': '{:.1f}'
    }).background_gradient(subset=['response_words'], cmap='Blues'))

    print(f"\nKey Insights:")
    print(f"- Average response length: {df_struct['response_words'].mean():.0f} words")
    print(f"- Vocabulary diversity ranges from {df_struct['vocab_diversity'].min():.3f} to {df_struct['vocab_diversity'].max():.3f}")
    print(f"- Highest repetition score: {df_struct['repetition_score'].max():.3f}")

In [ ]:
# Plot response length over steps with trend line
if 'df_struct' not in locals():
    print("⚠️ No structural data available - please run the analysis cells first")
else:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Structural Metrics Over Time', fontsize=16, y=1.02)

    # Response length
    ax1 = axes[0, 0]
    ax1.plot(df_struct['step'], df_struct['response_words'], 'o-', alpha=0.7)
    z = np.polyfit(df_struct['step'], df_struct['response_words'], 1)
    p = np.poly1d(z)
    ax1.plot(df_struct['step'], p(df_struct['step']), "r--", alpha=0.5)
    ax1.set_xlabel('Step')
    ax1.set_ylabel('Response Words')
    ax1.set_title('Response Length Trend')
    ax1.grid(True, alpha=0.3)

    # Vocabulary diversity
    ax2 = axes[0, 1]
    ax2.plot(df_struct['step'], df_struct['vocab_diversity'], 'o-', color='green', alpha=0.7)
    ax2.set_xlabel('Step')
    ax2.set_ylabel('Vocabulary Diversity')
    ax2.set_title('Vocabulary Diversity')
    ax2.grid(True, alpha=0.3)

    # Repetition score
    ax3 = axes[1, 0]
    ax3.plot(df_struct['step'], df_struct['repetition_score'], 'o-', color='red', alpha=0.7)
    ax3.set_xlabel('Step')
    ax3.set_ylabel('Repetition Score')
    ax3.set_title('Repetition Score (lower is better)')
    ax3.grid(True, alpha=0.3)

    # Depth vs response length
    ax4 = axes[1, 1]
    scatter = ax4.scatter(df_struct['depth'], df_struct['response_words'], 
                         c=df_struct['step'], cmap='viridis', s=100, alpha=0.7)
    ax4.set_xlabel('Depth')
    ax4.set_ylabel('Response Words')
    ax4.set_title('Response Length by Depth')
    plt.colorbar(scatter, ax=ax4, label='Step')
    ax4.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

## 2. Quality Signals Analysis

This section analyzes the linguistic and semantic quality of LLM responses. Each metric helps identify strengths, weaknesses, or patterns in the generated text.

### Key Metrics Explained

| Metric | Description | Why It Matters |
|--------|-------------|----------------|
| **Coherence Variance** | Measures how consistent the logical flow of a response is across steps. | High variance may indicate unstable or contradictory reasoning. |
| **Information Density** | Ratio of unique, meaningful content to total words. | High density suggests concise, informative responses. |
| **Specificity** | Degree to which responses include concrete details (e.g., numbers, names, examples). | Low specificity can signal generic or unhelpful outputs. |
| **Alignment** | How well responses match the user's intent or query. | Misalignment may reveal misunderstandings or biases in the LLM. |
| **Hedge Density** | Frequency of uncertain language (e.g., "might," "possibly"). | High density can indicate low confidence or overly cautious responses. |

In [ ]:
# Run quality signals analysis
print("Analyzing quality signals...")
quality_results = qs.analyze_quality_signals(
    jsonl_file, 
    use_embeddings=HAS_EMBEDDINGS, 
    use_spacy=HAS_SPACY
)
df_quality = pd.DataFrame(quality_results)

# Display quality metrics summary
display(Markdown("### Quality Metrics Summary"))
quality_cols = ['coherence_variance', 'info_density', 'specificity', 'alignment', 'hedge_density']
available_cols = [col for col in quality_cols if col in df_quality.columns]

if available_cols:
    display(df_quality[['step', 'node_id'] + available_cols].style.background_gradient(
        subset=available_cols, cmap='RdYlBu_r'
    ).format({col: '{:.3f}' for col in available_cols}))
else:
    print("No quality metrics available. Check if analysis ran correctly.")

In [ ]:
# Create comprehensive quality visualization
if 'coherence_variance' in df_quality.columns and 'info_density' in df_quality.columns:
    plt.figure(figsize=(12, 8))
    
    # Create scatter plot
    scatter = sns.scatterplot(
        x='coherence_variance',
        y='info_density',
        hue='step',
        data=df_quality,
        palette='viridis',
        s=100,
        alpha=0.7,
        edgecolor='black',
        linewidth=0.5
    )
    
    plt.title("Coherence vs. Information Density by Step", fontsize=14, pad=20)
    plt.xlabel("Coherence Variance (lower = better)", fontsize=12)
    plt.ylabel("Information Density (higher = better)", fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.3)
    
    # Add quadrants labels
    plt.text(0.02, 0.98, 'Ideal', transform=plt.gca().transAxes, 
             fontsize=12, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.3))
    plt.text(0.98, 0.02, 'Problematic', transform=plt.gca().transAxes, 
             fontsize=12, horizontalalignment='right',
             bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.3))
    
    # Annotate outliers
    for i, row in df_quality.iterrows():
        if row['coherence_variance'] > 0.8 or row['info_density'] < 0.2:
            plt.annotate(
                f"Step {row['step']}",
                (row['coherence_variance'], row['info_density']),
                fontsize=9,
                alpha=0.8,
                xytext=(5, 5),
                textcoords='offset points'
            )
    
    # Add colorbar
    norm = plt.Normalize(df_quality['step'].min(), df_quality['step'].max())
    sm = plt.cm.ScalarMappable(cmap='viridis', norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, label='Step')
    
    plt.tight_layout()
    plt.show()
    
    # Print insights
    print("\n### Quality Insights:")
    print(f"- Average coherence variance: {df_quality['coherence_variance'].mean():.3f}")
    print(f"- Average information density: {df_quality['info_density'].mean():.3f}")
    
    # Find best and worst responses
    best_idx = df_quality['info_density'].idxmax()
    worst_idx = df_quality['coherence_variance'].idxmax()
    
    print(f"\nMost informative response: Step {df_quality.loc[best_idx, 'step']} "
          f"(density: {df_quality.loc[best_idx, 'info_density']:.3f})")
    print(f"Least coherent response: Step {df_quality.loc[worst_idx, 'step']} "
          f"(variance: {df_quality.loc[worst_idx, 'coherence_variance']:.3f})")

### Actionable Insights

#### High Coherence Variance:
- Check if the LLM's context window or prompts need adjustment
- Example: Add constraints like "Respond in 3 bullet points" to reduce variability

#### Low Information Density:
- Use post-processing to filter repetitive phrases
- Prompt the LLM to "elaborate with examples"

#### High Hedge Density:
- Experiment with temperature settings
- Add "Be confident in your answer" to prompts

## 3. Convergence Analysis

Convergence analysis helps us understand if the LLM is stabilizing on a solution or drifting aimlessly. We look for plateaus (stable periods) and fixed points (exact repetitions).

In [ ]:
# Run convergence tracker
print("Running convergence analysis...")
summary = ct.analyze_convergence(jsonl_file, use_embeddings=HAS_EMBEDDINGS)
df_converge = pd.DataFrame(summary['per_node_results'])

# Display convergence summary
display(Markdown("### Convergence Summary"))
print(f"Total steps analyzed: {len(df_converge)}")
print(f"Average drift from root: {df_converge['drift_from_root'].mean():.3f}")
print(f"Number of plateaus detected: {len(summary['plateaus'])}")

if summary['fixed_point_step']:
    print(f"Fixed point reached at step: {summary['fixed_point_step']}")
else:
    print("No fixed point detected (responses kept changing)")

# Show plateau details
if summary['plateaus']:
    print("\nPlateau periods:")
    for i, (start, end) in enumerate(summary['plateaus'][:5], 1):
        print(f"  {i}. Steps {start}-{end} (duration: {end-start+1})")

In [ ]:
# Plot convergence with plateaus and fixed points
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Main convergence plot
ax1.plot(df_converge['step'], df_converge['drift_from_root'], 
         'o-', linewidth=2, markersize=6, alpha=0.8)

# Highlight plateaus
for i, (start, end) in enumerate(summary['plateaus']):
    ax1.axvspan(start, end, color='orange', alpha=0.2, label='Plateau' if i == 0 else '')
    # Add plateau label
    mid_point = (start + end) / 2
    max_drift = df_converge.loc[(df_converge['step'] >= start) & (df_converge['step'] <= end), 'drift_from_root'].max()
    ax1.text(mid_point, max_drift + 0.02, f'Plateau\n({end-start+1} steps)', 
             ha='center', va='bottom', fontsize=9)

# Fixed point line
if summary['fixed_point_step']:
    ax1.axvline(summary['fixed_point_step'], color='red', linestyle='--', 
                linewidth=2, label='Fixed Point')
    ax1.text(summary['fixed_point_step'], ax1.get_ylim()[1]*0.9, 
             'Fixed\nPoint', ha='center', va='top', fontsize=9,
             bbox=dict(boxstyle='round', facecolor='red', alpha=0.2))

ax1.set_xlabel('Step', fontsize=12)
ax1.set_ylabel('Drift from Root', fontsize=12)
ax1.set_title('Convergence Analysis: Drift Over Steps', fontsize=14, pad=20)
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)

# Drift gradient plot
if len(df_converge) > 2:
    gradients = np.gradient(df_converge['drift_from_root'])
    ax2.bar(df_converge['step'], gradients, alpha=0.7, color='purple')
    ax2.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    ax2.set_xlabel('Step', fontsize=12)
    ax2.set_ylabel('Drift Gradient', fontsize=12)
    ax2.set_title('Rate of Change (Gradient)', fontsize=14)
    ax2.grid(True, alpha=0.3)
    
    # Highlight significant changes
    threshold = np.std(gradients)
    significant = np.abs(gradients) > threshold
    if np.any(significant):
        ax2.scatter(df_converge.loc[significant, 'step'], 
                   gradients[significant], 
                   color='red', s=50, zorder=5, label='Significant Change')
        ax2.legend()

plt.tight_layout()
plt.show()

## 4. Interactive Exploration

Use the widgets below to explore the data. Filter by different metrics to find interesting patterns or problematic responses.

In [ ]:
# Interactive filtering dashboard
style = {'description_width': '120px'}
layout = widgets.Layout(width='400px')

# Create widgets for filtering
step_slider = widgets.IntRangeSlider(
    value=[0, len(df_nodes)-1],
    min=0,
    max=len(df_nodes)-1,
    step=1,
    description='Step Range:',
    style=style,
    layout=layout
)

rep_slider = widgets.FloatSlider(
    value=0.5,
    min=0,
    max=1.0,
    step=0.05,
    description='Max Repetition:',
    style=style,
    layout=layout
)

hedge_slider = widgets.FloatSlider(
    value=0.3,
    min=0,
    max=1.0,
    step=0.05,
    description='Max Hedge Density:',
    style=style,
    layout=layout
)

density_slider = widgets.FloatSlider(
    value=0.2,
    min=0,
    max=1.0,
    step=0.05,
    description='Min Info Density:',
    style=style,
    layout=layout
)

metric_dropdown = widgets.Dropdown(
    options=['Response Length', 'Vocabulary Diversity', 'Repetition Score', 
             'Coherence', 'Info Density', 'Hedge Density'],
    value='Response Length',
    description='Plot Metric:',
    style=style,
    layout=layout
)

# Output area
output = widgets.Output()

def update_dashboard(*args):
    with output:
        clear_output(wait=True)
        
        # Apply filters
        mask = (
            (df_struct['step'] >= step_slider.value[0]) &
            (df_struct['step'] <= step_slider.value[1]) &
            (df_struct['repetition_score'] <= rep_slider.value)
        )
        
        # Add quality filters if available
        if 'hedge_density' in df_quality.columns:
            mask = mask & (df_quality['hedge_density'] <= hedge_slider.value)
        if 'info_density' in df_quality.columns:
            mask = mask & (df_quality['info_density'] >= density_slider.value)
        
        filtered_struct = df_struct[mask]
        filtered_quality = df_quality[mask] if not df_quality.empty else pd.DataFrame()
        
        # Display summary
        display(Markdown(f"### Filtered Results: {len(filtered_struct)} responses"))
        
        if not filtered_struct.empty:
            # Create plot based on selected metric
            fig, ax = plt.subplots(figsize=(10, 4))
            
            metric_map = {
                'Response Length': 'response_words',
                'Vocabulary Diversity': 'vocab_diversity',
                'Repetition Score': 'repetition_score',
                'Coherence': 'coherence_variance',
                'Info Density': 'info_density',
                'Hedge Density': 'hedge_density'
            }
            
            selected_metric = metric_map[metric_dropdown.value]
            
            if selected_metric in filtered_struct.columns:
                ax.plot(filtered_struct['step'], filtered_struct[selected_metric], 
                       'o-', linewidth=2, markersize=6)
            elif selected_metric in filtered_quality.columns:
                ax.plot(filtered_quality['step'], filtered_quality[selected_metric], 
                       'o-', linewidth=2, markersize=6, color='orange')
            
            ax.set_xlabel('Step')
            ax.set_ylabel(metric_dropdown.value)
            ax.set_title(f'{metric_dropdown.value} Over Filtered Steps')
            ax.grid(True, alpha=0.3)
            plt.show()
            
            # Show table of extreme values
            display(Markdown("#### Extreme Values:"))
            if selected_metric in filtered_struct.columns:
                extreme = filtered_struct.nlargest(3, selected_metric) if 'Score' not in metric_dropdown.value else filtered_struct.nsmallest(3, selected_metric)
            else:
                extreme = filtered_quality.nlargest(3, selected_metric) if 'Score' not in metric_dropdown.value else filtered_quality.nsmallest(3, selected_metric)
            
            display(extreme[['step', 'node_id', selected_metric]].style.background_gradient(
                subset=[selected_metric], cmap='RdYlBu_r'
            ))

# Connect widgets to update function
for widget in [step_slider, rep_slider, hedge_slider, density_slider, metric_dropdown]:
    widget.observe(update_dashboard, 'value')

# Display the dashboard
display(Markdown("### Interactive Filtering Dashboard"))
display(widgets.VBox([
    widgets.HBox([step_slider, rep_slider]),
    widgets.HBox([hedge_slider, density_slider]),
    metric_dropdown,
    output
]))

# Initial update
update_dashboard()

### Export Filtered Data

Use this cell to export your filtered results for further analysis.

In [ ]:
# Export functionality
export_button = widgets.Button(description="Export to CSV", button_style='success')
filename_input = widgets.Text(value='filtered_results.csv', description='Filename:')

def export_data(b):
    # Apply current filters
    mask = (
        (df_struct['step'] >= step_slider.value[0]) &
        (df_struct['step'] <= step_slider.value[1]) &
        (df_struct['repetition_score'] <= rep_slider.value)
    )
    
    if 'hedge_density' in df_quality.columns:
        mask = mask & (df_quality['hedge_density'] <= hedge_slider.value)
    if 'info_density' in df_quality.columns:
        mask = mask & (df_quality['info_density'] >= density_slider.value)
    
    # Merge structural and quality data
    filtered_struct = df_struct[mask].copy()
    filtered_quality = df_quality[mask].copy() if not df_quality.empty else pd.DataFrame()
    
    if not filtered_quality.empty:
        merged = filtered_struct.merge(filtered_quality[['step', 'node_id'] + available_cols], 
                                      on=['step', 'node_id'], how='left')
    else:
        merged = filtered_struct
    
    # Export
    output_path = Path.cwd() / filename_input.value
    merged.to_csv(output_path, index=False)
    print(f"Exported {len(merged)} rows to {output_path}")

export_button.on_click(export_data)
display(widgets.HBox([filename_input, export_button]))

## 5. Summary & Next Steps

### What We Learned

Based on the analysis above, here are the key findings from this run:

In [ ]:
# Generate automatic summary
display(Markdown("### Run Summary"))

# Basic stats
total_responses = len(df_nodes)
avg_length = df_struct['response_words'].mean()
length_std = df_struct['response_words'].std()
avg_diversity = df_struct['vocab_diversity'].mean()
avg_repetition = df_struct['repetition_score'].mean()

print(f"📊 **Basic Statistics:**")
print(f"   - Total responses: {total_responses}")
print(f"   - Average length: {avg_length:.0f} ± {length_std:.0f} words")
print(f"   - Vocabulary diversity: {avg_diversity:.3f} on average")
print(f"   - Repetition score: {avg_repetition:.3f} on average")

# Quality insights
if not df_quality.empty:
    print(f"\n🎯 **Quality Insights:**")
    if 'coherence_variance' in df_quality.columns:
        coherence_trend = 'improving' if df_quality['coherence_variance'].iloc[-1] < df_quality['coherence_variance'].iloc[0] else 'degrading'
        print(f"   - Coherence is {coherence_trend} over time")
    if 'info_density' in df_quality.columns:
        density_trend = 'improving' if df_quality['info_density'].iloc[-1] > df_quality['info_density'].iloc[0] else 'degrading'
        print(f"   - Information density is {density_trend} over time")
    if 'hedge_density' in df_quality.columns:
        avg_hedge = df_quality['hedge_density'].mean()
        confidence = 'confident' if avg_hedge < 0.2 else 'cautious' if avg_hedge < 0.4 else 'very uncertain'
        print(f"   - LLM appears {confidence} (hedge density: {avg_hedge:.3f})")

# Convergence insights
print(f"\n🔄 **Convergence Patterns:**")
if summary['fixed_point_step']:
    print(f"   - ✅ Reached fixed point at step {summary['fixed_point_step']}")
else:
    print(f"   - ❌ No fixed point reached (responses kept evolving)")

if summary['plateaus']:
    longest_plateau = max(summary['plateaus'], key=lambda p: p[1] - p[0])
    plateau_duration = longest_plateau[1] - longest_plateau[0] + 1
    print(f"   - Found {len(summary['plateaus'])} plateaus, longest lasting {plateau_duration} steps")
else:
    print(f"   - No stable plateaus detected")

# Recommendations
print(f"\n💡 **Recommendations:**")

if avg_repetition > 0.3:
    print(f"   - High repetition detected. Consider lowering temperature or adding variety prompts.")

if length_std > avg_length * 0.5:
    print(f"   - High variability in response length. Consider adding length constraints.")

if not summary['fixed_point_step'] and len(df_nodes) > 10:
    print(f"   - Responses didn't converge. This might indicate good exploration or lack of focus.")

if avg_diversity < 0.5:
    print(f"   - Low vocabulary diversity. Try prompting for more varied expression.")

print(f"\n🔍 **Next Steps to Explore:**")
print(f"   1. Try with embeddings enabled for semantic analysis")
print(f"   2. Compare multiple runs to spot consistent patterns")
print(f"   3. Analyze specific branches that showed interesting behavior")
print(f"   4. Export data for custom visualizations or statistical tests")

### Try It Yourself!

Here are some ideas for further exploration:

1. **Compare Different Models**: Run the same prompt with different models and compare their convergence patterns
2. **Temperature Effects**: Analyze how temperature settings affect repetition and coherence
3. **Prompt Engineering**: Test different prompt strategies and see how they impact quality metrics
4. **Multi-Run Analysis**: Load multiple JSONL files and compare aggregate statistics
5. **Custom Metrics**: Add your own quality metrics based on domain-specific needs

### Share Your Findings!

If you discover interesting patterns or have suggestions for improving this analysis, please share them with the community. Your insights help make these tools better for everyone.

---

*Last updated: January 2026*